<h3> Análisis de migración de la margen de hielo en Nevados de Colombia 2000 - 2022 </h3>
<p><strong>Nombre: </strong> Juan Sebastián Hernández Santana </p>
<p><strong>Fecha: </strong> 06 de Enero de 2024 </p>

In [2]:
# Conexión con la versión desarrollador de Google Earth Engine
import ee
import geemap
# Manejo de datos
import pandas as pd
import numpy as np
# Manejo de gráficos
import matplotlib.pyplot as plt
import seaborn as sns
# Machine learning
from geemap import ml
from sklearn import ensemble

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [11]:
Map = geemap.Map(center = [-75.39, 4.75], zoom = 8)

Map = geemap.Map(basemap = 'HYBRID')
Map.add_basemap('OpenTopoMap')
Map.add_basemap('Esri.WorldImagery')

In [12]:
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<h3> Área de ínteres (Nevados, Purace) </h3>

# <font style="color:red">1. Colecciones y ROI </font>

In [19]:
roi =ee.Feature(
    ee.Geometry.Polygon([
        [[-74.481355,0.932196],
              [-74.481355,0.631389],
              [-74.412,0.631389],
              [-74.412,0.932196]]
    ]),
        {
          "area": "chiribiquete",
          "system:index": "0"
        })

In [20]:
#Visualizar la región de interés
Map.addLayer(roi,{},"roi")

In [21]:
#Colección de imagenes landsat 8
Imagenes = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterMetadata('CLOUD_COVER','less_than',30)
)

# <font style="color:green">1.1 Enmascarar nubes </font>

In [22]:
#Enmascaramiento de nubes
def maskL8sr(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 4)
    qa = img.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return img.updateMask(mask)

In [23]:
Imagenes_sinNubes = Imagenes.map(maskL8sr)

# <font style="color:green">1.2 Cortar imagenes </font>

In [24]:
#Cortar imagenes
def cortar(img):
    return img.clip(roi).copyProperties(img,["system:time_start"])

In [25]:
Imagenes_cortadas = Imagenes_sinNubes.map(cortar)

# <font style="color:green">1.3 Crear NDVI </font>

In [26]:
def toNDVI8(img):
    ndvi = img.expression('((NIR - RED) / (NIR + RED))',
    {'NIR': img.select('SR_B5'),
    'RED': img.select('SR_B4')}).rename('NDVI').copyProperties(img, ['system:time_start'])
    return img.addBands(ndvi)

In [27]:
Imagenes_cortadas = Imagenes_cortadas.map(toNDVI8)

# <font style="color:green">1.4 Crear imagenes promedio anuales </font>

In [28]:
# Imagenes promedio por años
LS2014 = Imagenes_cortadas.filterDate("2014-01-01","2014-12-31").mean()
LS2015 = Imagenes_cortadas.filterDate("2015-01-01","2015-12-31").mean()
LS2016 = Imagenes_cortadas.filterDate("2016-01-01","2016-12-31").mean()
LS2017 = Imagenes_cortadas.filterDate("2017-01-01","2017-12-31").mean()
LS2018 = Imagenes_cortadas.filterDate("2018-01-01","2018-12-31").mean()
LS2019 = Imagenes_cortadas.filterDate("2019-01-01","2019-12-31").mean()
LS2020 = Imagenes_cortadas.filterDate("2020-01-01","2020-12-31").mean()
LS2021 = Imagenes_cortadas.filterDate("2021-01-01","2021-12-31").mean()
LS2022 = Imagenes_cortadas.filterDate("2022-01-01","2022-12-31").mean()

# <font style="color:green">1.5 Mostrar promedio anuales </font>

In [29]:
# Imprimir el resultado en el visor
Map.addLayer(LS2014.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2014")
Map.addLayer(LS2015.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2015")
Map.addLayer(LS2016.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2016")
Map.addLayer(LS2017.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2017")
Map.addLayer(LS2018.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2018")
Map.addLayer(LS2019.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2019")
Map.addLayer(LS2020.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2020")
Map.addLayer(LS2021.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2021")
Map.addLayer(LS2022.select(["SR_B4","SR_B3","SR_B2"]), {"min":8256.6, "max":9647.4}, "2022")

# <font style="color:red">2. SELECCIÓN DE PUNTOS DE ENTRENAMIENTO </font>

# <font style="color:green">2.1 Año 2014 </font>

In [30]:
paleta = ['#8000ff', ' #7e03ff', ' #7c06ff', ' #7a09ff', ' #780dff', ' #7610ff', ' #7413ff', ' #7216ff', ' #7019ff', ' #6e1cff', ' #6c1fff', ' #6a22fe', ' #6826fe', ' #6629fe', ' #642cfe', ' #622ffe', ' #6032fe', ' #5e35fe', ' #5c38fd', ' #5a3bfd', ' #583efd', ' #5641fd', ' #5444fd', ' #5247fc', ' #504afc', ' #4e4dfc', ' #4c50fc', ' #4a53fb', ' #4856fb', ' #4659fb', ' #445cfb', ' #425ffa', ' #4062fa', ' #3e65fa', ' #3c68f9', ' #396bf9', ' #386df9', ' #3670f8', ' #3473f8', ' #3176f8', ' #3079f7', ' #2e7bf7', ' #2c7ef7', ' #2981f6', ' #2884f6', ' #2686f5', ' #2489f5', ' #218cf4', ' #208ef4', ' #1e91f3', ' #1c93f3', ' #1996f3', ' #1898f2', ' #169bf2', ' #149df1', ' #11a0f1', ' #10a2f0', ' #0ea5ef', ' #0ca7ef', ' #09a9ee', ' #08acee', ' #06aeed', ' #04b0ed', ' #01b3ec', ' #00b5eb', ' #02b7eb', ' #04b9ea', ' #07bbea', ' #08bee9', ' #0ac0e8', ' #0dc2e8', ' #0fc4e7', ' #10c6e6', ' #12c8e6', ' #14cae5', ' #17cbe4', ' #18cde4', ' #1acfe3', ' #1dd1e2', ' #1fd3e1', ' #20d5e1', ' #22d6e0', ' #24d8df', ' #27dade', ' #28dbde', ' #2adddd', ' #2ddedc', ' #2fe0db', ' #30e1da', ' #32e3da', ' #34e4d9', ' #37e6d8', ' #38e7d7', ' #3ae8d6', ' #3dead5', ' #3febd5', ' #40ecd4', ' #42edd3', ' #44eed2', ' #46efd1', ' #48f1d0', ' #4af2cf', ' #4df3ce', ' #4ef3cd', ' #50f4cc', ' #52f5cb', ' #54f6cb', ' #56f7ca', ' #58f8c9', ' #5af8c8', ' #5df9c7', ' #5efac6', ' #60fac5', ' #62fbc4', ' #64fbc3', ' #66fcc2', ' #68fcc1', ' #6afdc0', ' #6dfdbf', ' #6efebe', ' #70febc', ' #72febb', ' #74feba', ' #76ffb9', ' #78ffb8', ' #7affb7', ' #7dffb6', ' #7effb5', ' #80ffb4', ' #82ffb3', ' #84ffb2', ' #86ffb0', ' #88ffaf', ' #8bfeae', ' #8cfead', ' #8efeac', ' #90feab', ' #92fda9', ' #94fda8', ' #96fca7', ' #99fca6', ' #9bfba5', ' #9cfba4', ' #9efaa2', ' #a0faa1', ' #a2f9a0', ' #a4f89f', ' #a6f89d', ' #a8f79c', ' #abf69b', ' #acf59a', ' #aef498', ' #b0f397', ' #b2f396', ' #b4f295', ' #b6f193', ' #b9ef92', ' #bbee91', ' #bced8f', ' #beec8e', ' #c0eb8d', ' #c2ea8c', ' #c4e88a', ' #c6e789', ' #c8e688', ' #cbe486', ' #cce385', ' #cee184', ' #d0e082', ' #d2de81', ' #d4dd80', ' #d6db7e', ' #d9da7d', ' #dbd87b', ' #dcd67a', ' #ded579', ' #e0d377', ' #e2d176', ' #e4cf74', ' #e6cd73', ' #e8cb72', ' #ebca70', ' #ecc86f', ' #eec66d', ' #f0c46c', ' #f2c26b', ' #f4c069', ' #f6be68', ' #f9bb66', ' #fbb965', ' #fcb763', ' #feb562', ' #ffb360', ' #ffb05f', ' #ffae5e', ' #ffac5c', ' #ffa95b', ' #ffa759', ' #ffa558', ' #ffa256', ' #ffa055', ' #ff9d53', ' #ff9b52', ' #ff9850', ' #ff964f', ' #ff934d', ' #ff914c', ' #ff8e4a', ' #ff8c49', ' #ff8947', ' #ff8646', ' #ff8444', ' #ff8143', ' #ff7e41', ' #ff7b40', ' #ff793e', ' #ff763d', ' #ff733b', ' #ff703a', ' #ff6d38', ' #ff6b37', ' #ff6835', ' #ff6533', ' #ff6232', ' #ff5f30', ' #ff5c2f', ' #ff592d', ' #ff562c', ' #ff532a', ' #ff5029', ' #ff4d27', ' #ff4a26', ' #ff4724', ' #ff4422', ' #ff4121', ' #ff3e1f', ' #ff3b1e', ' #ff381c', ' #ff351b', ' #ff3219', ' #ff2f18', ' #ff2c16', ' #ff2914', ' #ff2613', ' #ff2211', ' #ff1f10', ' #ff1c0e', ' #ff190d', ' #ff160b', ' #ff1309', ' #ff1008', ' #ff0d06', ' #ff0905', ' #ff0603', ' #ff0302', ' #ff0000']

In [31]:
Map.addLayer(LS2014.select("NDVI"), {"min":0.2, "max":0.4, 'palette':paleta}, "NDVI 2014")

# <font style="color:blue">2.1.1 En esta opción vamos a traer los puntos que marquemos del visor </font>

In [32]:
#Nota: seleccionar primero los puntos en el visor y luego correr la linea
#Sin vegetación año 2014
SV2014 = ee.FeatureCollection(Map.draw_features)

In [33]:
SV2014

In [34]:
SV2014 = SV2014.map(lambda feature: feature.set('zona', 0))

In [35]:
SV2014

In [36]:
# Ahora vamos a seleccionar los punto que corresponden a la vegetación
# Nota: en la opción "delete layers" del visor borren los puntos de la clase pasada para empezar una nueva
CV2014 = ee.FeatureCollection(Map.draw_features)

In [37]:
CV2014

In [38]:
CV2014 = CV2014.map(lambda feature: feature.set('zona', 1))

In [39]:
CV2014

In [40]:
#Ahora vamos a unificar las dos clases que hemos creado en una sola colección
#Entrenamiento2014 =  ee.FeatureCollection(ee.List([SV2014,CV2014]))
#Entrenamiento2014 =  SV2014.merge(CV2014)
Entrenamiento2014 =  SV2014.merge(CV2014)

In [41]:
Entrenamiento2014

In [42]:
#Podemos revisar la primera colección "SV" empleando algunas de las propiedades que les asignamos
Entrenamiento2014.filter(ee.Filter.eq('zona',0))

In [43]:
#Podemos revisar la primera colección "SV" empleando algunas de las propiedades que les asignamos
Entrenamiento2014.filter(ee.Filter.eq('zona',1))

In [44]:
# En esta opción podemos guardar nuestros datos de entrenamiento como un archivo .shp

#geemap.ee_to_shp(Entrenamiento2014, "DatosEntrenamientoRF.shp")

Generating URL ...
Please wait ...
Data downloaded to C:\Users\valiz\Downloads\Maestría en Gestión de Información y Tecnologías Geoespaciales\Procesamiento Matemático y Digital de Imágenes\Clases\Procesamiento_Matematico_Digital_Imgs\DatosEntrenamientoRF.shp


# <font style="color:green">2.2 Bandas empleadas para la predicción </font>

In [45]:
bandas = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','NDVI']

In [46]:
#Etique que contiene las propiedades de los puntos creados
etiqueta = "zona"

# <font style="color:red">3. ENTRENAMIENTO </font>

In [47]:
# https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest#colab-python

# https://geemap.org/notebooks/32_supervised_classification/#add-a-legend-to-the-map

# https://www.youtube.com/playlist?list=PLLJHvBcuQLfWGHaUgtWYPu4mcyT6Wl3WR

In [48]:
entrenamiento = LS2014.select(bandas).sampleRegions(
    **{'collection': Entrenamiento2014, 'properties': [etiqueta], 'scale': 30}
)

In [75]:
entrenamiento

In [49]:
print(entrenamiento.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '1_0_0', 'properties': {'NDVI': 0.24617096781730652, 'SR_B1': 10070, 'SR_B2': 10584.333333333334, 'SR_B3': 13079.666666666666, 'SR_B4': 13000.333333333334, 'SR_B5': 21599.333333333332, 'SR_B6': 19320.333333333332, 'SR_B7': 15539, 'zona': 0}}


In [50]:
#Vamos a emplear Random Forest
#Opciones: numberOfTrees, variablesPerSplit, minLeafPopulation, bagFraction, maxNodes, seed

#numberOfTrees	Número de árboles a crear (número entero)
#variablesPerSplit	por defecto: nulo	El número de variables por división. Si no se especifica, utiliza la raíz cuadrada del número de variables.(entero)
#minLeafPopulation	por defecto: 1	Cree únicamente nodos cuyo conjunto de entrenamiento contenga al menos esta cantidad de puntos (entero)
#bagFraction	por defecto: 0.5	TLa fracción de entrada al bag por árbol (flotante)
#maxNodes	por defecto: nulo	El número máximo de nodos de hojas en cada árbol. Si no se especifica, el valor predeterminado es sin límite (entero)
#seed	por defecto: 0	La semilla de la aleatorización (entero)

DatosEntrenados = ee.Classifier.smileRandomForest(**{"numberOfTrees":100}).train(entrenamiento, etiqueta, bandas)

In [51]:
# Información de los datos entrenados por el clasificador
display('Resultados del entrenamiento con RF', DatosEntrenados.explain())

'Resultados del entrenamiento con RF'

In [52]:
ExactitudEntrenamiento = DatosEntrenados.confusionMatrix()

In [53]:
display('Matriz de confusión', ExactitudEntrenamiento)

'Matriz de confusión'

In [54]:
display('Exactitud general del entrenamiento', ExactitudEntrenamiento.accuracy())

'Exactitud general del entrenamiento'

# <font style="color:red">4. CLASIFICACIÓN </font>

In [55]:
resultados = LS2014.select(bandas).classify(DatosEntrenados)

In [56]:
resultados

In [57]:
Map.addLayer(resultados.randomVisualizer(), {}, 'coverturas 2014')

# <font style="color:red">5. CALCULAR ÁREAS </font>

In [58]:
#Área de mi poligono en km^2

roi.area().divide(1000000)

In [59]:
#Área clasificada como deforestada (0) y con vegetación (1) en km^2

df = geemap.image_area_by_group(
    resultados, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

Calculating area for group 0 ...
Calculating area for group 1 ...


,area,percentage
group,,
0,111.3191,0.4337
1,145.3626,0.5663


# <font style="color:red">6. REALIZAR RF CON LOS DATOS DE ENTRENAMIENTO DESCARGADOS </font>

In [60]:
PuntoEntrenamiento = geemap.shp_to_ee("DatosEntrenamientoRF.shp")

In [61]:
PuntoEntrenamiento

In [83]:
Muestraentrenamiento = LS2014.select(bandas).sampleRegions(
    **{'collection': PuntoEntrenamiento, 'properties': [etiqueta], 'scale': 30}
)

In [84]:
print(Muestraentrenamiento.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'NDVI': 0.24617096781730652, 'SR_B1': 10070, 'SR_B2': 10584.333333333334, 'SR_B3': 13079.666666666666, 'SR_B4': 13000.333333333334, 'SR_B5': 21599.333333333332, 'SR_B6': 19320.333333333332, 'SR_B7': 15539, 'zona': 0}}


In [85]:
#Vamos a tomar una muestra de cada clase de los puntos de entrenamiento que creamos

muestra = Muestraentrenamiento.randomColumn() #Entre más puntos de entrenamiento tengamos es mejor

In [86]:
muestraEntrenamiento = muestra.filter('random <= 0.8') #Tomamos el 80% de los datos para entrenar el modelo

In [87]:
muestraValidacion = muestra.filter('random > 0.8') #Tomamos el 20% de los datos para validar el modelo

In [88]:
muestraEntrenamiento

In [89]:
muestraValidacion

In [90]:
# Entrenar el clasificador con 10 árboles

EntrenamientoRF = ee.Classifier.smileRandomForest(**{"numberOfTrees":10}).train(muestraEntrenamiento,etiqueta,bandas)

In [91]:
#Proyectamos los resultados del entrenamiento para el modelo RF

display('Resultados del entrenamiento', EntrenamientoRF.explain())

'Resultados del entrenamiento'

In [92]:
#Matriz de confusión del entrenamiento

exactitudEntrenamiento = EntrenamientoRF.confusionMatrix()

In [93]:
display('Matriz de confusión del entrenamiento', exactitudEntrenamiento)

'Matriz de confusión del entrenamiento'

In [94]:
display('Exactitud global del entrenamiento', exactitudEntrenamiento.accuracy())

'Exactitud global del entrenamiento'

In [95]:
# Ahora vamos a realizar la validación, esto se hace ya con los datos entrenados

validacionDeDatos = muestraValidacion.classify(EntrenamientoRF)

In [97]:
exactitudValidacion = validacionDeDatos.errorMatrix(etiqueta, 'classification')

In [98]:
display('Matriz de confusión de la validacion', exactitudValidacion)

'Matriz de confusión de la validacion'

In [99]:
display('Exactitud de la validación', exactitudValidacion.accuracy())

'Exactitud de la validación'

In [102]:
#Vamos a clasificar nuevamente nuestra imagen

img_clasificada_2014 = LS2014.select(bandas).classify(EntrenamientoRF)

In [104]:
img_clasificada_2014

In [106]:
Map.addLayer(img_clasificada_2014.select("classification"), {}, 'coverturas 2014 segunda prueba')

In [107]:
Map

Map(bottom=130765.8354097678, center=[0.8324111011089462, -74.32246728176072], controls=(WidgetControl(options…

In [112]:
#geemap.ee_export_image(img_clasificada_2014,"Cobertura2014.tif",scale=30,crs=None,crs_transform=None,region=ee.Geometry.Polygon([
#        [[-74.481355,0.932196],
#              [-74.481355,0.631389],
#              [-74.412,0.631389],
#              [-74.412,0.932196]]
#    ]),dimensions=None,file_per_band=False,)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\valiz\Downloads\Maestría en Gestión de Información y Tecnologías Geoespaciales\Procesamiento Matemático y Digital de Imágenes\Clases\Procesamiento_Matematico_Digital_Imgs\Cobertura2014.tif
